Thesis Work.Using  Bert and FARM

In [ ]:
!git clone https://github.com/deepset-ai/FARM.git
!pip install -r FARM/requirements.txt
!pip install FARM/

Cloning into 'FARM'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 6438 (delta 49), reused 43 (delta 26), pack-reused 6352
Receiving objects: 100% (6438/6438), 65.76 MiB | 23.57 MiB/s, done.
Resolving deltas: 100% (4808/4808), done.
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 748.8MB 25kB/s 
     |████████████████████████████████| 47.7MB 62kB/s 
     |████████████████████████████████| 890kB 39.3MB/s 
     |████████████████████████████████| 327kB 41.7MB/s 
     |████████████████████████████████| 2.5MB 22.0MB/s 
     |████████████████████████████████| 163kB 43.0MB/s 
     |████████████████████████████████| 1.1MB 36.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 9.4MB/s 
     |█████████████████

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wEKLoEcimSzgcM7IrTanzqbNW9hBbvgjivGS0FoTTJM4HUcyyhj_-w
Mounted at /content/drive


In [ ]:
!git clone https://github.com/guggio/bbc_news

Cloning into 'bbc_news'...
remote: Enumerating objects: 2203, done.
remote: Counting objects: 100% (2203/2203), done.
remote: Compressing objects: 100% (2177/2177), done.
remote: Total 2203 (delta 28), reused 2189 (delta 18), pack-reused 0
Receiving objects: 100% (2203/2203), 5.18 MiB | 4.20 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [ ]:
from farm.data_handler.data_silo import DataSilo
from farm.data_handler.processor import TextClassificationProcessor
from farm.modeling.optimization import initialize_optimizer
from farm.infer import Inferencer
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import MultiLabelTextClassificationHead
from farm.modeling.tokenization import Tokenizer
from farm.train import Trainer
from farm.utils import set_all_seeds, MLFlowLogger, initialize_device_settings
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from pathlib import Path

df = pd.read_csv('/content/drive/My Drive/Hate/NewFileTD.txt',sep='\t', encoding='utf-8',names=["text", "label"])

#df.to_csv('/content/drive/My Drive/FinalProperTwoCdata',sep='\t')
train, test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
# determine the path where to save the train and test file
train_path = Path('/content/drive/My Drive/Hate/Train/', 'train.tsv')
test_path = Path('/content/drive/My Drive/Hate/Test/', 'test.tsv')

# save the train and test file
# again using the '\t' separator to create tab-separated-values files
train.to_csv(train_path )
test.to_csv(test_path)


In [ ]:
ml_logger = MLFlowLogger(tracking_uri="https://public-mlflow.deepset.ai/")
ml_logger.init_experiment(experiment_name="ADRs", run_name="Thesis Work")


 __          __  _                            _        
 \ \        / / | |                          | |       
  \ \  /\  / /__| | ___ ___  _ __ ___   ___  | |_ ___  
   \ \/  \/ / _ \ |/ __/ _ \| '_ ` _ \ / _ \ | __/ _ \ 
    \  /\  /  __/ | (_| (_) | | | | | |  __/ | || (_) |
     \/  \/ \___|_|\___\___/|_| |_| |_|\___|  \__\___/ 
  ______      _____  __  __  
 |  ____/\   |  __ \|  \/  |              _.-^-._    .--.
 | |__ /  \  | |__) | \  / |           .-'   _   '-. |__|
 |  __/ /\ \ |  _  /| |\/| |          /     |_|     \|  |
 | | / ____ \| | \ \| |  | |         /               \  |
 |_|/_/    \_\_|  \_\_|  |_|        /|     _____     |\ |
                                     |    |==|==|    |  |
|---||---|---|---|---|---|---|---|---|    |--|--|    |  |
|---||---|---|---|---|---|---|---|---|    |==|==|    |  |
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 


In [ ]:
set_all_seeds(seed=42)
device, n_gpu = initialize_device_settings(use_cuda=True)
n_epochs = 5
batch_size = 16 # larger batch sizes might use too much computing power in Colab
evaluate_every = 100

09/10/2020 15:33:41 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None


In [ ]:
lang_model = "bert-base-cased"
do_lower_case = False

tokenizer = Tokenizer.load(
    pretrained_model_name_or_path=lang_model,
    do_lower_case=do_lower_case)

NameError: ignored

In [ ]:
label_list = ['Text','Label'] #labels in our data set
metric = "f1_macro" # desired metric for evaluation

processor = TextClassificationProcessor(tokenizer=tokenizer,
                                            max_seq_len=512, # BERT can only handle sequence lengths of up to 512
                                            data_dir='/content/drive/My Drive/Hate/FinalDS', 
                                            label_list=label_list,
                                            label_column_name="label", # our labels are located in the "genre" column
                                            metric=metric,
                                            quote_char='"',
                                            multilabel=True,
                                            train_filename="train.tsv",
                                            dev_filename=None,
                                            test_filename="test.tsv",
                                            dev_split=0.1 # this will extract 10% of the train set to create a dev set
                                            )

In [ ]:
data_silo = DataSilo(
    processor=processor,
    batch_size=8)

09/10/2020 15:34:26 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
09/10/2020 15:34:26 - INFO - farm.data_handler.data_silo -   Loading train set from: /content/drive/My Drive/Hate/FinalDS/train.tsv 


ValueError: ignored

In [ ]:
# loading the pretrained BERT base cased model
language_model = LanguageModel.load(lang_model)
# prediction head for our model that is suited for classifying news article genres
prediction_head = MultiLabelTextClassificationHead(num_labels=len(label_list))

model = AdaptiveModel(
        language_model=language_model,
        prediction_heads=[prediction_head],
        embeds_dropout_prob=0.1,
        lm_output_types=["per_sequence"],
        device=device)

model, optimizer, lr_schedule = initialize_optimizer(
        model=model,
        learning_rate=3e-5,
        device=device,
        n_batches=8,
        n_epochs=n_epochs)

09/10/2020 15:13:19 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
09/10/2020 15:13:19 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 2]
09/10/2020 15:13:22 - INFO - farm.modeling.optimization -   Loading optimizer `TransformersAdamW`: '{'correct_bias': False, 'weight_decay': 0.01, 'lr': 3e-05}'
09/10/2020 15:13:24 - INFO - farm.modeling.optimization -   Using scheduler 'get_linear_schedule_with_warmup'
09/10/2020 15:13:24 - INFO - farm.modeling.optimization -   Loading schedule `get_linear_schedule_with_warmup`: '{'num_warmup_steps': 4.0, 'num_training_steps': 40}'


In [ ]:
trainer = Trainer(
        model=model,
        optimizer=optimizer,
        data_silo=data_silo,
        epochs=n_epochs,
        n_gpu=n_gpu,
        lr_schedule=lr_schedule,
        evaluate_every=evaluate_every, # we defined this value in the setup section. We set it to 100
        device=device)

NameError: ignored

In [ ]:
trainer.train()

07/29/2020 17:36:36 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

Evaluating: 100%|██████████| 97/97 [00:26<00:00,  3.66it/s]
07/29/2020 17:38:23 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EV

AdaptiveModel(
  (language_model): Bert(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(28996, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
            

In [ ]:
save_dir = "saved_models/bert-english-news-article"
model.save(save_dir)
processor.save(save_dir)

inferenced_model = Inferencer.load(save_dir)

In [ ]:
!zip -r saved_models/model.zip saved_models/bert-english-news-article

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from pathlib import Path

df = pd.read_csv('/content/drive/My Drive/FinalProperTwoCdata/mytsv4.tsv',sep='\t')

#df.to_csv('/content/drive/My Drive/FinalProperTwoCdata',sep='\t')
train, test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

# determine the path where to save the train and test file
train_path = Path('/content/drive/My Drive/FinalProperTwoCdata/', 'train.tsv')
test_path = Path('/content/drive/My Drive/FinalProperTwoCdata/', 'test.tsv')

# save the train and test file
# again using the '\t' separator to create tab-separated-values files
train.to_csv(train_path, sep='\t', index=False)
test.to_csv(test_path, sep='\t', index=False)

FileNotFoundError: ignored